### EDA

In [1]:
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import chardet
from sklearn.model_selection import KFold, train_test_split, cross_val_score 
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import roc_auc_score, auc, roc_curve
import warnings
warnings.filterwarnings('ignore')

data_root = "./data/"


In [2]:
dfTrain = pd.read_csv(os.path.join(data_root, 'train_offline.csv'))
dfTrain.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN


In [3]:
dfTest = pd.read_csv(os.path.join(data_root, 'test_offline.csv'))
dfTest.head() # It belongs to classification problem due to label values

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,to_testset
0,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN,0,True
1,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN,0,True
2,1439408,2632,NaN,NaN,0.0,NaN,20160516.0,-1,True
3,1439408,2632,8591.0,20:1,0.0,20160516.0,20160613.0,0,True
4,2029232,450,1532.0,30:5,0.0,20160530.0,NaN,0,True


In [4]:
dfTrain.shape

(1160742, 7)

In [5]:
dfTest.shape

(594142, 9)

In [6]:
dfTrain.info() # Data has only float, int and object
               # No variable column has null/missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1160742 entries, 0 to 1160741
Data columns (total 7 columns):
User_id          1160742 non-null int64
Merchant_id      1160742 non-null int64
Coupon_id        746969 non-null float64
Discount_rate    746969 non-null object
Distance         1090916 non-null float64
Date_received    746969 non-null float64
Date             456709 non-null float64
dtypes: float64(4), int64(2), object(1)
memory usage: 62.0+ MB


In [7]:
dfTest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594142 entries, 0 to 594141
Data columns (total 9 columns):
User_id          594142 non-null int64
Merchant_id      594142 non-null int64
Coupon_id        306313 non-null float64
Discount_rate    306313 non-null object
Distance         557965 non-null float64
Date_received    306313 non-null float64
Date             320275 non-null float64
label            594142 non-null int64
to_testset       594142 non-null bool
dtypes: bool(1), float64(4), int64(3), object(1)
memory usage: 36.8+ MB


In [8]:
dfTrain.describe() # check outlier

,User_id,Merchant_id,Coupon_id,Distance,Date_received,Date
count,1.160742e+06,1.160742e+06,746969.000000,1.090916e+06,7.469690e+05,4.567090e+05
mean,3.690549e+06,4.103626e+03,7090.106124,2.726183e+00,2.016022e+07,2.016029e+07
std,2.123784e+06,2.364912e+03,4159.042350,3.687066e+00,1.142962e+02,1.117400e+02
min,4.000000e+00,1.000000e+00,4.000000,0.000000e+00,2.016010e+07,2.016010e+07
25%,1.845197e+06,2.146000e+03,3304.000000,0.000000e+00,2.016013e+07,2.016020e+07
50%,3.697362e+06,3.532000e+03,7610.000000,1.000000e+00,2.016020e+07,2.016032e+07
75%,5.532561e+06,6.412000e+03,10323.000000,4.000000e+00,2.016032e+07,2.016040e+07
max,7.361032e+06,8.856000e+03,14045.000000,1.000000e+01,2.016043e+07,2.016063e+07


In [17]:
dfTest.label.unique() #Target variable/Dependent variable is discrete and categorical in nature
                      #Label range from 0 to 1

array([ 0, -1,  1], dtype=int64)

In [12]:
dfTest.Distance.describe() #check relationship between label and distance

count    557965.000000
mean          1.648883
std           2.919379
min           0.000000
25%           0.000000
50%           0.000000
75%           2.000000
max          10.000000
Name: Distance, dtype: float64

In [10]:
dfTest.label.value_counts() # label has most values in -1 and 1

-1    287829
 0    278222
 1     28091
Name: label, dtype: int64

In [5]:
f = open(os.path.join(data_root, 'column_description.csv'), 'rb')
print(chardet.detect(f.read(100))) # check encoding
dfColDesc = pd.read_csv(os.path.join(data_root, 'column_description.csv'), encoding='big5')
dfColDesc.head()

{'encoding': 'Big5', 'confidence': 0.99, 'language': 'Chinese'}


,Column,Description
0,User_id,用戶 ID
1,Merchant_id,商家 ID
2,Coupon_id,優惠券 ID (null 代表無優惠券消費)
3,Discount_rate,"優惠券折價：[0,1] 代表折扣率；x:y 代表滿 x 減 y 元"
4,Distance,"用戶經常活動地點離商家最近距離 (x * 500 公尺), 0 表示低於 500 公尺, 1..."


In [6]:
dfColDescEn = pd.read_csv(os.path.join(data_root, 'column_description_en.csv'))
dfColDescEn.head()

,Column,Description
0,User_id,User id
1,Merchant_id,shop/merchant id
2,Coupon_id,"Coupon id. If it is null, it means the transac..."
3,Discount_rate,"Discount rate, [0,1] means ratio of discount. ..."
4,Distance,Distance between users' most active regions an...


In [7]:
dfTest["label"].value_counts()

-1    287829
 0    278222
 1     28091
Name: label, dtype: int64

In [8]:
def label(row):
    if np.isnan(row["Date_received"]):
        return -1
    

dfTrain["label"] = dfTrain.apply(label)
dfTrain["label"].value_counts()


KeyError: ('Date_received', 'occurred at index User_id')